# Overview
This notebook compares the predicted solar PV generation profiles at each existing REIPPPP site.
The aggregated fleet production is then validated against actual Eskom data from https://www.eskom.co.za/dataportal/.

Solar resource data is sourced from:
 - ERA5
   - ERA5 data can be downloaded through the Copernicus Climate Data Store cdsapi package, and requires 
registration and setup as per https://cds.climate.copernicus.eu/api-how-to. (See notebook prepare_atlite_cutouts.ipynb)
 - SARAH 
   - SARAH data can be downloaded via from EUMETSAT CMSAF using the portal https://wui.cmsaf.eu/safira/action/viewProduktSearch
Surface Incoming Direct Radiation (SID), Surface Incoming Shortwave Radiation (SIS) and Direct Normal Irradiance (DNI) should be downloaded
and extracted into the sarah_dir path. (See notebook prepare_atlite_cutouts.ipynb)
- NREL NSRDB
   - The NREL National Solar Resource Database (NSRDB) can be accessed through either a range of options
      - Via developer API (see https://developer.nrel.gov/docs/solar/nsrdb/)
      - Through System Advisor Model (SAM) software
      - Via the NREL Highly Scalable Data Service (see https://github.com/NREL/hsds-examples)

   - The NSRDB data used in this notebook was accessed through SAM. A series (lat,lon) points were generated at the center of each ERA5 grid cell
and then imported into SAM. The csv file for the ERA5 grid points can be found under XXX

- CSIR and Fraunhofer Solar Wind Aggregation Study for South Africa

Recreating data using atlite and ERA5 is simpler than using atlite with SARAH or using NSRDB. Both python libraries include PV models for fixed and tracking configuorations.

The raw solar resource data is converted into power generation profiles using 2 python libraries.
- Atlite (see https://atlite.readthedocs.io/en/latest/)
- PySAM by NREL (see https://nrel-pysam.readthedocs.io/en/main/)


In [1]:
import atlite
import xarray as xr
import pandas as pd
import PySAM.Pvwattsv8 as pv
import numpy as np
import os
import matplotlib.pyplot as plt

from shapely import Point
import geopandas as gpd

from _helpers import (
    get_nsrdb_weather_file,
    aggregate_intra_region,
    generate_pv_timeseries,
    reshape_xarray,
    load_gis_data,
    find_closest_nsrdb_file,
)

# Load input data

In [2]:
# load REIPPPP plant data
reippp_data = pd.read_csv('reipppp_solar_data.csv', index_col=1)
reippp_data["ignore"] = reippp_data["COD"].isna()
reippp_data["COD"] = pd.to_datetime(reippp_data["COD"])

In [3]:
data_bundle_path = "../../data/bundle"
gis_data = load_gis_data(data_bundle_path)

Loading Eskom Supply Regions from ../../data/bundle/rsa_supply_regions.gpkg
Loading EIA applications from ../../data/bundle/REEA_OR_2023_Q3.shp
Loading REDZs from ../../data/bundle/REDZs.shp
Loading Power Corridors from ../../data/bundle/Power_corridors.shp
Loading SACAD from ../../data/bundle/SACAD_OR_2023_Q3.shp
Loading SAPAD from ../../data/bundle/SAPAD_OR_2023_Q3.shp
Loading SKA exclusion from ../../data/bundle/SKA/SKA_exclusion.shp


# atlite ERA5 and SARAH

In [4]:
# load cutouts - already downloaded
cutout_era5 = atlite.Cutout(
    path="../../cutouts/RSA-2010_22-era5.nc",
    chunks={"time": 1000},
)
cutout_era5.data = cutout_era5.data.sel(time=slice("2017-01-01", "2022-12-31"))
cutout_era5.data = cutout_era5.data.sel(time=~((cutout_era5.data.time.dt.month == 2) & (cutout_era5.data.time.dt.day == 29)))

era5_ft = generate_pv_timeseries(cutout_era5, "Fixed Tilt", dc_ac_ratio=1, module="era5") #dc-ac ratio implemented later in code, set as 1 here
era5_sat = generate_pv_timeseries(cutout_era5, "Single Axis", dc_ac_ratio=1, module="era5") #dc-ac ratio implemented later in code, set as 1 here
era5_pu_ds = xr.Dataset({"Fixed Tilt": era5_ft, "Single Axis": era5_sat})

cutout_sarah = atlite.Cutout(
    path="../../cutouts/RSA-2017_22-sarah.nc",
    chunks={"time": 1000},
)
cutout_sarah.data = cutout_sarah.data.sel(time=slice("2017-01-01", "2022-12-31"))
cutout_sarah.data = cutout_sarah.data.sel(time=~((cutout_sarah.data.time.dt.month == 2) & (cutout_sarah.data.time.dt.day == 29)))

cutout_sarah.data = cutout_sarah.data.interp(y=cutout_era5.grid.y.unique(), x=cutout_era5.grid.x.unique(), method="linear")
cutout_sarah = cutout_sarah.merge(cutout_era5, compat="override")

sarah_ft = generate_pv_timeseries(cutout_sarah, "Fixed Tilt", dc_ac_ratio=1, module="sarah") #dc-ac ratio implemented later in code, set as 1 here
sarah_sat = generate_pv_timeseries(cutout_sarah, "Single Axis", dc_ac_ratio=1, module="sarah") #dc-ac ratio implemented later in code, set as 1 here
sarah_pu_ds = xr.Dataset({"Fixed Tilt": sarah_ft, "Single Axis": sarah_sat})

[########################################] | 100% Completed | 6.57 s
[########################################] | 100% Completed | 7.93 s
[########################################] | 100% Completed | 15.93 s
[########################################] | 100% Completed | 18.29 s


In [5]:
era5_power = pd.DataFrame(
    0,
    index = cutout_era5.coords["time"].values,
    columns = reippp_data.index
)
capacity = era5_power.copy()
sarah_power = era5_power.copy()

era5_pu = era5_power.copy()
sarah_pu = era5_power.copy()

for idx, row in reippp_data.iterrows():
  
    era5_pu[idx] = era5_pu_ds[row["Type"]].sel(lat=row["latitude"], lon=row["longitude"], method="nearest").to_series()
    era5_pu[idx] = (era5_pu[idx] * row["dc_ac_ratio"]).clip(upper=1)
    
    instl_cap = pd.Series(row["capacity"], index = era5_pu.index)
    instl_cap[instl_cap.index < row["COD"]] = 0
    capacity[idx] = instl_cap
    if row["ignore"]:
        capacity[idx] = 0
    era5_power[idx] = era5_pu[idx] * capacity[idx]

    sarah_pu[idx] = sarah_pu_ds[row["Type"]].sel(lat=row["latitude"], lon=row["longitude"], method="nearest").to_series()
    sarah_pu[idx] = (sarah_pu[idx] * row["dc_ac_ratio"]).clip(upper=1)
    sarah_power[idx] = sarah_pu[idx] * capacity[idx]

In [6]:
# remove any 29 Feb on leap years
era5_power = era5_power[~((era5_power.index.month == 2) & (era5_power.index.day == 29))]
sarah_power = sarah_power[~((sarah_power.index.month == 2) & (sarah_power.index.day == 29))]
era5_pu = era5_pu[~((era5_pu.index.month == 2) & (era5_pu.index.day == 29))]
sarah_pu = sarah_pu[~((sarah_pu.index.month == 2) & (sarah_pu.index.day == 29))]

# CSIR and Fraunhofer SWA Study

In [7]:
# Only available for RSA as single region or as 27 Supply Regions
csir_fise = pd.read_excel("csir_fise_SWA_data.xlsx", sheet_name = "27-solar_pv", index_col=0, parse_dates=True)
csir_fise = csir_fise.iloc[1:]
csir_fise.index = pd.DatetimeIndex(csir_fise.index)
csir_fise = csir_fise[~((csir_fise.index.month == 2) & (csir_fise.index.day == 29))]

In [8]:
points_gdf = [Point(lon, lat) for lon,lat in reippp_data[["longitude", "latitude"]].values]
points_gdf = gpd.GeoDataFrame(geometry=points_gdf, crs="EPSG:4326")
region_list = gpd.sjoin(points_gdf,gis_data["supply_regions"][27],how="left",op="within")["name"]
reippp_data["csir_region"] = region_list.values

# Only 1 year is available in CSIR-FISE data which gets repeated to fill up according to the REIPPPP data
csir_pu = pd.DataFrame(
    index = era5_pu.index,
    columns = reippp_data.index
)

for idx, row in reippp_data.iterrows():
    for y in range(2017, 2023):
        csir_pu.loc[csir_pu.index.year == y, idx] = csir_fise[row["csir_region"]].values

/home/pklein/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


# NSRDB and PySAM
This code assumes that data points for each reippp site have been downloaded from the available NREL platforms

In [9]:
sam_path = "../../data/weather/NREL SAM weather files/REIPPPP/"

pv_model = pv.new()
pv_inputs_ft ={
		"use_wf_albedo" : 1,
		"system_capacity" : 120000,
		"module_type" : 1,
		"dc_ac_ratio" : 1,
		"bifaciality" : 0,
		"array_type" : 0,
		"azimuth" : 0,
		"gcr" : 0.1,
		"soiling" : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
		"losses" : 11.42,
		"en_snowloss" : 0,
		"inv_eff" : 96,
		"batt_simple_enable" : 0,
		"constant" : 0
	}
pv_inputs_sat={
	"use_wf_albedo" : 1,
	"system_capacity" : 120000,
	"module_type" : 1,
	"dc_ac_ratio" : 1.15,
	"bifaciality" : 0,
	"array_type" : 2,
	"tilt" : 0,
	"azimuth" : 0,
	"gcr" : 0.3,
	"soiling" : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
	"losses" : 11.42, 
	"en_snowloss" : 0,
	"inv_eff" : 96,
	"batt_simple_enable" : 0,
	"constant" : 0
}

In [10]:
sam_files = os.listdir(sam_path)

In [11]:
# run through all gpd coordinates and get the capacity factor for each
pysam_power = pd.DataFrame(index = pd.date_range('2017-01-01 00:00','2019-12-31 23:00',freq='H'),columns=reippp_data.index)
pysam_pu = pysam_power.copy()
for idx, row in reippp_data.iterrows():
    if not row["ignore"]:

        for y in [2017,2018,2019]:
            # Try load file with lat,lon directly else find closest available file
            try:
                lat = row["latitude"]
                lon = row["longitude"]
                path = f"{sam_path}{lat}_{lon}"
                file = get_nsrdb_weather_file(path, 60, y)    
            except:
               lat,lon = find_closest_nsrdb_file((lat,lon), sam_files)
               path = f"{sam_path}{lat}_{lon}"
               file = get_nsrdb_weather_file(path, 60, y)        

            if row['Type'] =='Fixed Tilt':
                pv_inputs = pv_inputs_ft.copy()
                pv_inputs['tilt'] = -row['latitude']
                if row['BW'] <=2:
                    pv_inputs["module_type"]=0
            else:
                pv_inputs = pv_inputs_sat.copy()

            pv_inputs['solar_resource_file'] = f"{os.getcwd()}/{path}/{file}"
            pv_inputs['dc_ac_ratio'] = row['dc_ac_ratio']

            # iterate through the input key-value pairs and set the module inputs
            for k, v in pv_inputs.items():
                if k != 'number_inputs':
                    pv_model.value(k, v)

            # run the module
            pv_model.execute()
            pysam_pu.loc[str(y),idx] = (pd.Series(pv_model.Outputs.ac)/(pv_inputs['system_capacity']/pv_inputs['dc_ac_ratio']*1000)).values

        instl_cap = pd.Series(row["capacity"], index = pysam_pu.index)
        instl_cap[instl_cap.index < row["COD"]] = 0
        capacity[idx] = instl_cap
        pysam_power[idx] = pysam_pu[idx] * capacity[idx]

pysam_power = pysam_power.fillna(0)
pysam_pu = pysam_pu.fillna(0)

/media/pklein/Data/Shared Github/pypsa-za2/pre_processing/resource_processing/_helpers.py:220: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  points_gdf["distance"] = points_gdf.distance(reippp_point)
/media/pklein/Data/Shared Github/pypsa-za2/pre_processing/resource_processing/_helpers.py:220: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  points_gdf["distance"] = points_gdf.distance(reippp_point)
/media/pklein/Data/Shared Github/pypsa-za2/pre_processing/resource_processing/_helpers.py:220: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  points_gdf["distance"] = points_gdf.

# Export data to NetCDF

In [12]:
for module in ["era5","sarah","csir"]:

    timeseries = xr.DataArray(
        np.zeros((len(era5_pu), len(era5_pu.columns), 2)),
        coords = {"time": era5_pu.index, "plant": era5_pu.columns.values, "param": ["power", "pu"]},
        dims = ["time", "plant", "param"]
    )
    if module == "era5":
        timeseries.loc[dict(plant=era5_pu.columns.values, param="pu")] = era5_pu.values
        timeseries.loc[dict(plant=era5_pu.columns.values, param="power")] = era5_power.values
    elif module == "sarah":
        timeseries.loc[dict(plant=sarah_pu.columns.values, param="pu")] = sarah_pu.values
        timeseries.loc[dict(plant=sarah_pu.columns.values, param="power")] = sarah_power.values
    elif module=="csir":
        timeseries.loc[dict(plant=csir_pu.columns.values, param="pu")] = csir_pu.values
    else:
        timeseries.loc[dict(plant=pysam_pu.columns.values, param="pu")] = pysam_pu.values
        timeseries.loc[dict(plant=pysam_pu.columns.values, param="power")] = pysam_power.values

    timeseries.to_netcdf(f"timeseries_data/fixed_solar_pv_{module}.nc")